In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel("C:\\Users\\Aswathi\\Desktop\\Riverdale_location_1.xlsx",
                   sheet_name = 'Edges',skiprows=0,header=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24886 entries, 0 to 24885
Data columns (total 52 columns):
Vertex 1                          24886 non-null object
Vertex 2                          24886 non-null object
Color                             0 non-null float64
Width                             0 non-null float64
Style                             0 non-null float64
Opacity                           0 non-null float64
Visibility                        0 non-null float64
Label                             0 non-null float64
Label Text Color                  0 non-null float64
Label Font Size                   0 non-null float64
Reciprocated?                     24886 non-null object
ID                                24886 non-null int64
Dynamic Filter                    0 non-null float64
Add Your Own Columns Here         0 non-null float64
Relationship                      24886 non-null object
Relationship Date (UTC)           24886 non-null datetime64[ns]
Tweet             

In [5]:
df = df[['Place Country Code','Place Full Name','Tweet']]

In [6]:
df.sample(5)

,Place Country Code,Place Full Name,Tweet
9190,NaN,NaN,RT @CW_Riverdale: This crime must be atoned fo...
11093,NaN,NaN,RT @mia__mcintyre: see if Archie off riverdale...
16961,NaN,NaN,RT @WriterRAS: The #Serpents are back in next ...
6334,NaN,NaN,RT @willo719: Posted vs tagged photo\n\n#River...
17342,NaN,NaN,Riverdale 🤔🤔


In [7]:
df[['City','state']] = df['Place Full Name'].str.split(',',n =1,expand=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24886 entries, 0 to 24885
Data columns (total 5 columns):
Place Country Code    357 non-null object
Place Full Name       357 non-null object
Tweet                 24886 non-null object
City                  357 non-null object
state                 349 non-null object
dtypes: object(5)
memory usage: 972.2+ KB


In [9]:
df.sample(7)

,Place Country Code,Place Full Name,Tweet,City,state
14778,NaN,NaN,Tou parva com este episódio de riverdale,NaN,NaN
4612,NaN,NaN,RT @Longliveolicity: On #Riverdale is Hal rea...,NaN,NaN
2485,NaN,NaN,que pibe lindo el colorado de riverdale,NaN,NaN
10623,NaN,NaN,Mom: So I've been looking at some condos in Ri...,NaN,NaN
8849,NaN,NaN,RT @RiverdaleBRNews: ESTÁVEL! \n\nO episódio 2...,NaN,NaN
7242,NaN,NaN,RT @madelainepetsch: Running away from ur prob...,NaN,NaN
16927,NaN,NaN,@cmiIapower vai ver riverdale,NaN,NaN


In [10]:
df.drop('Place Full Name', axis = 1, inplace=True)

In [11]:
df.columns = ['Country','Tweet','City','State']

In [12]:
df = df[df['Tweet'].isnull()==False]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24886 entries, 0 to 24885
Data columns (total 4 columns):
Country    357 non-null object
Tweet      24886 non-null object
City       357 non-null object
State      349 non-null object
dtypes: object(4)
memory usage: 972.1+ KB


## Cleaning the tweets to remove
1. URL links
2. HTML tags
3. Numbers
4. Hashtags and Mentions(@)
5. Emoticons

In [14]:
import re
import csv
from textblob import TextBlob
from bs4 import BeautifulSoup
from tweepy import OAuthHandler
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [15]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aswathi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

In [17]:
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', text)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)# creates unnecessary white spaces
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case) #tokenize and join together to remove unneccessary white spaces
    return (" ".join(words)).strip()

In [18]:
testing = df.Tweet[:10]
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))
test_result

['rockets win final score of today s varsity softball game rockridge riverdale this brings our record to softball sports da',
 'rockets win final score of today s varsity softball game rockridge riverdale this brings our record to softball sports da',
 'rockets win final score of today s varsity softball game rockridge riverdale this brings our record to softball sports da',
 'rockets win final score of today s varsity softball game rockridge riverdale this brings our record to softball sports da',
 'revamped account oc not new to rp or riverdale fandom multiship down to rp as long as you re literate no smut info backstory in tweet below collage credit to',
 'field girls unseat riverdale to win first county championship',
 'field girls unseat riverdale to win first county championship',
 'field girls unseat riverdale to win first county championship',
 'e a riverdalians quem quer ganhar uma caneca de riverdale em comemora o aos k do cmbr estaremos sorteando esta caneca m a r a da para 

In [19]:
len(df.Tweet)

24886

In [20]:
df['Tweet'][8786]

'RT @camzxlili: Just catching up on yesterday’s episode with the pupper\n\n@colesprouse @lilireinhart @CamilaMendes @kj_apa  @CW_Riverdale #ri…'

In [22]:
%%time
print("Cleaning and parsing the tweets...\n")
clean_tweet = []
for i in range(0,len(df.Tweet)):  
    #print(i)
    #print(df['Tweet'][i])
    clean_tweet.append(tweet_cleaner(df['Tweet'][i]))

Cleaning and parsing the tweets...

Wall time: 8.4 s


In [23]:
clean_tweet

['rockets win final score of today s varsity softball game rockridge riverdale this brings our record to softball sports da',
 'rockets win final score of today s varsity softball game rockridge riverdale this brings our record to softball sports da',
 'rockets win final score of today s varsity softball game rockridge riverdale this brings our record to softball sports da',
 'rockets win final score of today s varsity softball game rockridge riverdale this brings our record to softball sports da',
 'revamped account oc not new to rp or riverdale fandom multiship down to rp as long as you re literate no smut info backstory in tweet below collage credit to',
 'field girls unseat riverdale to win first county championship',
 'field girls unseat riverdale to win first county championship',
 'field girls unseat riverdale to win first county championship',
 'e a riverdalians quem quer ganhar uma caneca de riverdale em comemora o aos k do cmbr estaremos sorteando esta caneca m a r a da para 

In [24]:
clean_df = pd.DataFrame(clean_tweet,columns=['cleaned_tweet'])

In [25]:
df_clean = pd.concat([df,clean_df], axis = 1, ignore_index = True)

In [26]:
df.head()

,Country,Tweet,City,State
0,NaN,Rockets Win! Final score of today’s varsity so...,NaN,NaN
1,NaN,Rockets Win! Final score of today’s varsity so...,NaN,NaN
2,NaN,Rockets Win! Final score of today’s varsity so...,NaN,NaN
3,NaN,Rockets Win! Final score of today’s varsity so...,NaN,NaN
4,NaN,Revamped Account!\n\n~ OC\n\n~ Not new to RP o...,NaN,NaN


In [27]:
df_clean.columns = ['Country','Tweet','City','State','Clean_tweet']

In [28]:
df_clean.sample(5)

,Country,Tweet,City,State,Clean_tweet
20049,NaN,An iconic #Riverdale scene 😍 https://t.co/bpad...,NaN,NaN,an iconic riverdale scene
20373,NaN,@torimicaletti @Georginaa088 @RT_Riverdale @CW...,NaN,NaN,riverdale riverdale he promised her safety tho...
9827,NaN,RT @lilireinhart: Facial expressions #Riverdal...,NaN,NaN,rt facial expressions riverdale
23218,NaN,"Happy Birthday to #Riverdale's Toni Topaz, @Va...",NaN,NaN,happy birthday to riverdale s toni topaz enjoy...
21840,NaN,@nckyrmchnd @CW_Riverdale what epi ka na ba? h...,NaN,NaN,riverdale what epi ka na ba hahaha


## Performing Sentiment Analysis using Text Blob

In [29]:
for i in range(0,len(df_clean)):
    blob = TextBlob(df_clean.loc[i,"Clean_tweet"])
    df_clean.loc[i,'polarity'] = blob.sentiment.polarity

In [30]:
df_clean.sample(5)

,Country,Tweet,City,State,Clean_tweet,polarity
1492,NaN,@torimicaletti @Georginaa088 @RT_Riverdale @CW...,NaN,NaN,riverdale riverdale but also have hal and alic...,0.0
1449,NaN,RT @aliciaareal_: quero a 2º temporada de rive...,NaN,NaN,rt quero a temporada de riverdale na netflix p j,0.0
13693,NaN,Estoy obsesionado con el episodio musical de R...,NaN,NaN,estoy obsesionado con el episodio musical de r...,0.0
4893,NaN,RT @WriterRAS: The #Serpents are back in next ...,NaN,NaN,rt the serpents are back in next week s riverd...,0.0
3782,NaN,RT @sprousehartftw: Okay but will Hiram name h...,NaN,NaN,rt okay but will hiram name his prison riverja...,0.5


In [31]:
for i in range(0,len(df_clean)):
    if(df_clean.loc[i,"polarity"] > 0) :
        df_clean.loc[i,'sentiment'] = 1
    else:
        if(df_clean.loc[i,"polarity"] < 0):
            df_clean.loc[i,'sentiment'] = -1
        else:
            df_clean.loc[i,'sentiment'] = 0

In [32]:
df_clean.sample(5)

,Country,Tweet,City,State,Clean_tweet,polarity,sentiment
22057,NaN,RT @joyarianna: @colesprouse #jugheadjones #so...,NaN,NaN,rt jugheadjones southsideserpents riverdale,0.00000,0.0
6451,NaN,RT @AlexieRosenblum: You know what Alice deser...,NaN,NaN,rt you know what alice deserves these riverdale,0.00000,0.0
15825,NaN,RT @RiverdaleBrasil: A Sombra de uma Dúvida – ...,NaN,NaN,rt a sombra de uma d vida o novo cap tulo de r...,0.00000,0.0
16491,NaN,RT @shaymsquad: fandom de pretty little liars ...,NaN,NaN,rt fandom de pretty little liars falando pro f...,0.03125,1.0
18503,NaN,Je shippe trop Alice et FP. Encore et toujours...,NaN,NaN,je shippe trop alice et fp encore et toujours ...,0.00000,0.0


## Sentiment Analysis by Country

In [33]:
df_clean.nunique()

Country           28
Tweet          11410
City             221
State             75
Clean_tweet    10683
polarity         770
sentiment          3
dtype: int64

In [34]:
country_senti = df_clean.groupby(['Country'])['sentiment'].mean()
country_pol = df_clean.groupby(['Country'])['polarity'].mean()

In [35]:
country_senti.sample(5)

Country
AR    0.0
NZ    0.0
PT    0.0
TH    0.0
IE    0.5
Name: sentiment, dtype: float64

In [36]:
country_pol.sample(5)

Country
AU   -0.204630
KW    0.333333
PH    0.043089
DE    0.250000
MX    0.000000
Name: polarity, dtype: float64

In [65]:
df_clean.to_csv('riverdalesenti.csv',sep=',',index=False,encoding='utf-8')

## Finding percentage of negative vs positive tweets

In [54]:
pos_tweets = [ tweet for index, tweet in enumerate(df_clean['Clean_tweet'][df_clean['sentiment'] == 1])]
neu_tweets = [ tweet for index, tweet in enumerate(df_clean['Clean_tweet'][df_clean['sentiment'] == 0])]
neg_tweets = [ tweet for index, tweet in enumerate(df_clean['Clean_tweet'][df_clean['sentiment'] == -1])]

In [55]:
print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(df_clean['Clean_tweet'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(df_clean['Clean_tweet'])))
print("Percentage of negative tweets: {}%".format(len(neg_tweets)*100/len(df_clean['Clean_tweet'])))

Percentage of positive tweets: 31.403198585550108%
Percentage of neutral tweets: 55.087197621152455%
Percentage of negative tweets: 13.509603793297437%


## Sentiment Analysis for States in USA

In [56]:
df_state = df_clean[df_clean.Country == 'US']

In [57]:
df_state.shape

(81, 7)

In [58]:
df_state.nunique()

Country         1
Tweet          69
City           47
State          24
Clean_tweet    68
polarity       32
sentiment       3
dtype: int64

In [59]:
State_senti = df_state.groupby(['State'])['sentiment'].mean()
State_pol = df_state.groupby(['State'])['polarity'].mean()

In [60]:
State_senti.sample(5)

State
 Atlanta   -1.00
 PA         1.00
 MN        -1.00
 GA         0.25
 OH         0.00
Name: sentiment, dtype: float64

In [61]:
State_pol.sample(5)

State
 MN   -0.600000
 OK    0.700000
 MD   -0.138889
 GA    0.055000
 PA    0.125000
Name: polarity, dtype: float64

In [63]:
pos_tweets = [ tweet for index, tweet in enumerate(df_state['Clean_tweet'][df_state['sentiment'] == 1])]
neu_tweets = [ tweet for index, tweet in enumerate(df_state['Clean_tweet'][df_state['sentiment'] == 0])]
neg_tweets = [ tweet for index, tweet in enumerate(df_state['Clean_tweet'][df_state['sentiment'] == -1])]

In [64]:
print("Percentage of positive tweets within US: {}%".format(len(pos_tweets)*100/len(df_state['Clean_tweet'])))
print("Percentage of neutral tweets within US: {}%".format(len(neu_tweets)*100/len(df_state['Clean_tweet'])))
print("Percentage of negative tweets within US: {}%".format(len(neg_tweets)*100/len(df_state['Clean_tweet'])))

Percentage of positive tweets within US: 37.03703703703704%
Percentage of neutral tweets within US: 43.20987654320987%
Percentage of negative tweets within US: 19.753086419753085%
